In [12]:
from tensorflow.keras.preprocessing.text import one_hot

In [13]:
sent=[  'the glass of milk',
     'the glass of juice',
     'the cup of tea',
    'I am a good boy',
     'I am a good developer',
     'understand the meaning of words',
     'your videos are good']

In [14]:
max_length = max(len(line.split()) for line in sent)

print("Length of the longest statement:", max_length)

Length of the longest statement: 5


In [15]:
### Vocabulary size
voc_size=500

In [16]:
onehot_repr=[one_hot(words,voc_size) for words in sent] 
# This means convert ever word into the vocabulary size in the list sentences
print(onehot_repr)      

[[122, 156, 407, 375], [122, 156, 407, 355], [122, 384, 407, 131], [200, 113, 248, 276, 373], [200, 113, 248, 276, 273], [384, 122, 195, 407, 474], [390, 132, 462, 276]]


In [17]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [18]:
import numpy as np

Here what we do is we make padding of 8 cloumns add zeros before those statemnets whose length is less than 8 

In [19]:
## pre padding
sent_length=8
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

print(embedded_docs)

[[  0   0   0   0 122 156 407 375]
 [  0   0   0   0 122 156 407 355]
 [  0   0   0   0 122 384 407 131]
 [  0   0   0 200 113 248 276 373]
 [  0   0   0 200 113 248 276 273]
 [  0   0   0 384 122 195 407 474]
 [  0   0   0   0 390 132 462 276]]


In [20]:
## 10 feature dimesnions
dim=10


In [21]:
model=Sequential()
model.add(Embedding(voc_size,10,input_length=sent_length))
model.compile('adam','mse')

In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 8, 10)             5000      
                                                                 
Total params: 5000 (19.53 KB)
Trainable params: 5000 (19.53 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
model.predict(embedded_docs[0])

1/1 [==============================] - 0s 241ms/step


array([[-0.01959975,  0.04320956, -0.01745384,  0.02387864,  0.00182556,
        -0.0195815 , -0.02523458, -0.01730455,  0.01000396,  0.03102596],
       [-0.01959975,  0.04320956, -0.01745384,  0.02387864,  0.00182556,
        -0.0195815 , -0.02523458, -0.01730455,  0.01000396,  0.03102596],
       [-0.01959975,  0.04320956, -0.01745384,  0.02387864,  0.00182556,
        -0.0195815 , -0.02523458, -0.01730455,  0.01000396,  0.03102596],
       [-0.01959975,  0.04320956, -0.01745384,  0.02387864,  0.00182556,
        -0.0195815 , -0.02523458, -0.01730455,  0.01000396,  0.03102596],
       [-0.01495215, -0.02277734,  0.0267272 , -0.02839311,  0.00587813,
        -0.00629707,  0.02879622,  0.02426207,  0.01792837,  0.04449237],
       [ 0.03708472,  0.01626802,  0.03649267, -0.04434643, -0.00225797,
        -0.03882967, -0.0119549 , -0.00034777, -0.04284225, -0.02714298],
       [-0.03497501, -0.04269165, -0.0306077 , -0.04832708,  0.03537735,
        -0.03582592,  0.02543852,  0.0354864 

In [24]:
print(model.predict(embedded_docs)[0])

1/1 [==============================] - 0s 51ms/step
[[-0.01959975  0.04320956 -0.01745384  0.02387864  0.00182556 -0.0195815
  -0.02523458 -0.01730455  0.01000396  0.03102596]
 [-0.01959975  0.04320956 -0.01745384  0.02387864  0.00182556 -0.0195815
  -0.02523458 -0.01730455  0.01000396  0.03102596]
 [-0.01959975  0.04320956 -0.01745384  0.02387864  0.00182556 -0.0195815
  -0.02523458 -0.01730455  0.01000396  0.03102596]
 [-0.01959975  0.04320956 -0.01745384  0.02387864  0.00182556 -0.0195815
  -0.02523458 -0.01730455  0.01000396  0.03102596]
 [-0.01495215 -0.02277734  0.0267272  -0.02839311  0.00587813 -0.00629707
   0.02879622  0.02426207  0.01792837  0.04449237]
 [ 0.03708472  0.01626802  0.03649267 -0.04434643 -0.00225797 -0.03882967
  -0.0119549  -0.00034777 -0.04284225 -0.02714298]
 [-0.03497501 -0.04269165 -0.0306077  -0.04832708  0.03537735 -0.03582592
   0.02543852  0.0354864   0.01051297  0.02916663]
 [ 0.00307452 -0.01775594  0.03380981  0.04068496  0.03805834  0.02497998
   